# <span style="color:green"> Import libraries<span style="color:green">

In [1]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

2024-05-17 13:15:55.249102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 13:15:55.249170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 13:15:55.250400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 13:15:55.260350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 13:15:56.279396: W tensorflow/compiler/tf2

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [2]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/Imputeted_data/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

     FID  IID  PAT  MAT  SEX  PHENOTYPE  rs11253478_C  rs2436029:269101:C:T_T  \
0  10004    1    0    0    2         75             2                       2   
1  10005    1    0    0    1         69             2                       2   
2  10007    1    0    0    1        108             2                       2   
3  10008    1    0    0    1         94             1                       2   
4  10009    1    0    0    1         92             2                       2   

   rs2448378_T  rs2060138_A  ...  rs11137278_C  rs6559242_G  rs1110146_T  \
0            2            1  ...             0            2            2   
1            2            2  ...             2            0            2   
2            2            2  ...             2            1            1   
3            2            1  ...             2            1            2   
4            2            2  ...             2            0            1   

   rs6559257_A  rs7865267_C  rs1378954:141005656:G:A_A  

In [3]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (1282, 150125)
Shape of y (1282,)


In [4]:
y

0        75
1        69
2       108
3        94
4        92
       ... 
1277    174
1278     89
1279     77
1280     41
1281     96
Name: PHENOTYPE, Length: 1282, dtype: int64

In [5]:
#Train test data split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=2000)
print("The dimension of X_train is {}".format(X_train.shape))
print("The dimension of X_test is {}".format(X_test.shape))

The dimension of X_train is (897, 150125)
The dimension of X_test is (385, 150125)


In [6]:
#Standardization of X_train and X_test data by Standard Scaler
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

## <span style="color:green"> LASSO <span style="color:green">

In [7]:
# Create and fit the LASSOCV model select 5000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.00062], cv=5, random_state=2000)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.057e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.817e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.00062
5000 selected SNPs
print shape of selected snps by lasso (1282, 5000)


### Linear regression

In [8]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(l1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
lasso_LR_sigSNP
lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
lasso_LR_sig_col

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)
r2 is  0.23303138085329378
mean_absolute_error 14.699557894419062
mean_squared_error  340.5430770142561
explained_variance_score 0.6843293027084176
Selected p-values:
[2.01676452e-04 1.43949853e-04 4.38703206e-04 3.21844145e-04
 9.87836007e-06 3.59090408e-04 2.82080791e-04 2.30512913e-04
 4.15229151e-04 8.81470583e-05 3.83731430e-04 2.60801415e-04
 3.16260510e-04 2.92275193e-04 3.00579096e-04 2.10538444e-04
 5.40026007e-05 1.53164088e-04 2.76729956e-04 9.00736338e-05
 4.47945643e-05 3.21715747e-04 8.62655316e-05 1.93547690e-04
 2.59327281e-04 9.53972096e-06 2.12254426e-04 3.32753155e-04
 4.71543554e-05 3.60610538e-04 4.61565241e-04 5.48284776e-05]
Number of selected features: 32
Selected F-values:
[13.89567645 14.5373109  12.42442843 13.00950171 19.69236686 12.80245387
 13.2591343  13.64190431 12.5281647  15.47377989 12.67707877 13.40775707
 13.04261236 13.19189851 13.13885838 13.8139839  16.412850

['rs10692334:8486910:C:CTT_CTT',
 'rs1412922_G',
 'rs2463148:122215516:G:C_C',
 'rs1545693_T',
 'rs2921336:70680871:C:T_T',
 'rs10895555_A',
 'rs1543933_T',
 'rs7973648_G',
 'rs12708799:15687854:A:T_T',
 'rs12709295_G',
 'rs7218430_C',
 'rs271645_A',
 'rs17655474:76495486:C:G_G',
 'rs663178_G',
 'rs12606093_T',
 'rs2335629_C',
 'rs1290943_G',
 'rs741283_G',
 'rs735931_T',
 'rs9848819:74123221:C:T_T',
 'rs10937477_G',
 'rs9877345_A',
 'rs4690001_G',
 'rs4054037_A',
 'rs10434313_A',
 'rs17583962_C',
 'rs1705392_C',
 'rs4976445_A',
 'rs12531826_C',
 'rs749538_T',
 'rs3860832_G',
 'rs4736162_G']

### Random Forest

In [9]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -61.32219859003727
mean_absolute_error 26.107939565289787
mean_squared_error  1022.7131461065229
explained_variance_score 0.053657661042911586


### Support Vector Machine

In [10]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.5091640763071101
mean_absolute_error 14.318335398926672
mean_squared_error  319.1810746441081
explained_variance_score 0.7042333593187404


### XGBOOST

In [11]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -10.44686615458688
mean_absolute_error 26.652027932699625
mean_squared_error  1112.8437869098063
explained_variance_score -0.0317069362126432


## <span style="color:green"> Ridge<span style="color:green">

In [12]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [108972  11284  94133 ...  17089 134652  72090]
5000 selected SNPs
Maximum value of coefficient 0.09102136750922467
print shape of selected snps by Ridge (1282, 5000)


### Linear Regression

In [13]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [14]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(R1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
Ridge_LR_sigSNP
Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
Ridge_LR_sig_col

r2 is  0.7549066637586196
mean_absolute_error 10.274421623556924
mean_squared_error  160.69797683601143
explained_variance_score 0.8507279185220166
Selected p-values:
[9.53972096e-06 9.87836007e-06 9.00736338e-05 1.93547690e-04
 3.21715747e-04 2.81399239e-04 4.61565241e-04 4.02337062e-04
 2.92275193e-04 4.71543554e-05 2.82080791e-04 3.83214248e-04
 3.83731430e-04 2.94186599e-04 2.01676452e-04 4.60821220e-04
 3.00579096e-04 8.62655316e-05 2.60801415e-04 8.58780621e-05
 2.59327281e-04 3.32753155e-04 2.06875977e-04 4.85307366e-04
 3.59090408e-04 1.53164088e-04 3.99750776e-04 4.12958057e-04
 2.72387786e-04 2.30512913e-04 4.15229151e-04 5.63300490e-05]
Number of selected features: 32
Selected F-values:
[19.76006143 19.69236686 15.4324209  13.97385957 13.01025656 13.2637165
 12.3286534  12.58768808 13.19189851 16.67333484 13.2591343  12.67962567
 12.67707877 13.17955606 13.89567645 12.33169412 13.13885838 15.51506052
 13.40775707 15.52367418 13.41850213 12.94645322 13.84731701 12.23413791
 1

['rs17583962_C',
 'rs2921336:70680871:C:T_T',
 'rs9848819:74123221:C:T_T',
 'rs4054037_A',
 'rs9877345_A',
 'rs950968_G',
 'rs3860832_G',
 'rs6089240_C',
 'rs663178_G',
 'rs12531826_C',
 'rs1543933_T',
 'rs12505610:171302543:G:T_T',
 'rs7218430_C',
 'rs41421445_C',
 'rs10692334:8486910:C:CTT_CTT',
 'rs11946808_A',
 'rs12606093_T',
 'rs4690001_G',
 'rs271645_A',
 'rs815583_T',
 'rs10434313_A',
 'rs4976445_A',
 'rs4919870_C',
 'rs13267681_T',
 'rs10895555_A',
 'rs741283_G',
 'rs228516_T',
 'rs9882701_A',
 'rs10868511_T',
 'rs7973648_G',
 'rs12708799:15687854:A:T_T',
 'rs147731462:142598726:A:G_G']

### Random Forest

In [15]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -54.97620720460577
mean_absolute_error 25.76589827826682
mean_squared_error  1002.4261863259691
explained_variance_score 0.075142883876264


### Support Vector Machine

In [16]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)


imp_score=perm_importance.importances_mean
feature_indices = np.argsort(np.abs(imp_score))[::-1]
print(f"Selected feature indices are {feature_indices}")

# # Specify the number of features you want to select based on magnitude
num_selected_features = 100
top_100=feature_indices[:num_selected_features]
print(f"Top 100 fearure indices are {feature_indices}")
imp_score_100=imp_score[top_100]
print(f"Top 100 fearures permutation scores are {imp_score_100}")

Ridge_SVR_sigSNP= R1.iloc[:,top_100]
Ridge_SVR_sig_col= list(Ridge_SVR_sigSNP.columns)
Ridge_SVR_sig_col

r2 is  0.7851991165907694
mean_absolute_error 10.647254491629846
mean_squared_error  171.3323010367937
explained_variance_score 0.8408442684719689
Selected feature indices are [ 453 4670  582 ... 3719 3521 4867]
Top 100 fearure indices are [ 453 4670  582 ... 3719 3521 4867]
Top 100 fearures permutation scores are [ 0.00135808  0.00118914  0.0011486   0.00113551  0.00111324  0.00109869
  0.00109789  0.00107073  0.00104309  0.00102265  0.00102138  0.00098674
  0.00093106  0.0009192   0.00091295  0.00091067  0.00089312  0.00088733
  0.00088486  0.00087776  0.00087263  0.00087115  0.00086704  0.0008651
 -0.00086444  0.00086329  0.00086205  0.00085873  0.00085678  0.0008484
 -0.00084743 -0.00084332 -0.00084152  0.00082874  0.00082802  0.00082798
 -0.00082038  0.00081875  0.00081845  0.00079776  0.00079566 -0.00078964
 -0.00078592  0.00077974  0.00077974  0.00077883 -0.0007786  -0.00077189
 -0.00077159  0.00076795  0.0007673  -0.0007638   0.00075543  0.0007534
  0.00075242 -0.00075111  0.00

['rs6651479_T',
 'rs1377374_G',
 'rs2361019:22729613:T:A_A',
 'rs8037591:93702884:C:T_T',
 'rs11209761:71764175:T:C_C',
 'rs852027:17082332:G:C_C',
 'rs13258014_A',
 'rs3770911_G',
 'rs1047721_T',
 'rs627857_A',
 'rs135067_C',
 'rs5759855_T',
 'rs228288:36926181:T:A_A',
 'rs7992848_G',
 'rs2254544_G',
 'rs2875149_C',
 'rs8026827_G',
 'rs9516597_A',
 'rs4823615_A',
 'rs1012534_A',
 'rs1648168:109923392:A:G_G',
 'rs34692537:24542339:AT:A_A',
 'rs1327665_C',
 'rs3867263_A',
 'rs1181996:26505806:G:C_C',
 'rs6480502:73007215:T:A_A',
 'rs6754583:31007792:C:G_G',
 'rs741283_G',
 'rs2833440_A',
 'rs11084919_C',
 'rs2944540_A',
 'rs1549288:81060761:T:C_C',
 'rs6597545_G',
 'rs1744065_G',
 'rs10425874_A',
 'rs10746257_A',
 'rs10983088_G',
 'rs1903863_C',
 'rs396693_T',
 'rs2952518_G',
 'rs675257_G',
 'rs17719860_C',
 'rs7231686_A',
 'rs6665853_G',
 'rs10230715_T',
 'rs10184249:34546822:G:A_A',
 'rs264114_T',
 'rs4339899:55474476:C:G_G',
 'rs62365662:95801967:T:C_C',
 'rs16882852_C',
 'rs7452823_

### XGBoost

In [17]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -6.266538067944728
mean_absolute_error 27.09141607903815
mean_squared_error  1123.1760152126865
explained_variance_score -0.03338318094447423


## <span style="color:green"> Elastic net<span style="color:green">

In [18]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.045], cv=5, random_state= 2000)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.742e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.666e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.045
Optimal l1_ratio:  0.5
5000 selected SNPs
print shape of selected snps by Elastic net (1282, 5000)


### Linear Regression

In [19]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [20]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(E1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
Elasticnet_LR_sigSNP
Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
Elasticnet_LR_sig_col

r2 is  0.8837170121582624
mean_absolute_error 7.2949537183920246
mean_squared_error  78.98733507654364
explained_variance_score 0.9266095503165738
Selected p-values:
[2.01676452e-04 4.38703206e-04 3.21844145e-04 9.87836007e-06
 3.59090408e-04 2.82080791e-04 2.30512913e-04 4.15229151e-04
 4.23459355e-04 8.81470583e-05 3.83731430e-04 2.60801415e-04
 3.16260510e-04 2.92275193e-04 3.00579096e-04 2.06875977e-04
 2.10538444e-04 1.48493427e-04 3.99750776e-04 1.26153756e-04
 1.88917447e-06 4.02337062e-04 3.86573098e-04 2.81399239e-04
 1.96069811e-04 4.73626876e-04 5.40026007e-05 1.53164088e-04
 7.51106623e-06 2.76729956e-04 9.00736338e-05 9.63650962e-05
 4.12958057e-04 4.20318472e-04 4.39223499e-04 4.47945643e-05
 3.21715747e-04 8.62655316e-05 1.93547690e-04 4.60821220e-04
 1.26692210e-04 3.83214248e-04 2.59327281e-04 9.53972096e-06
 5.59941263e-05 2.12254426e-04 3.32753155e-04 4.13214069e-04
 8.58780621e-05 1.43274401e-04 2.82846263e-04 2.94186599e-04
 4.71543554e-05 4.14577984e-04 3.18303962

['rs10692334:8486910:C:CTT_CTT',
 'rs2463148:122215516:G:C_C',
 'rs1545693_T',
 'rs2921336:70680871:C:T_T',
 'rs10895555_A',
 'rs1543933_T',
 'rs7973648_G',
 'rs12708799:15687854:A:T_T',
 'rs1974492_G',
 'rs12709295_G',
 'rs7218430_C',
 'rs271645_A',
 'rs17655474:76495486:C:G_G',
 'rs663178_G',
 'rs12606093_T',
 'rs4919870_C',
 'rs2335629_C',
 'rs2207074:63800969:A:G_G',
 'rs228516_T',
 'rs6428517_A',
 'rs12034420_A',
 'rs6089240_C',
 'rs9983361_A',
 'rs950968_G',
 'rs11684629:12727091:G:C_C',
 'rs7580273_G',
 'rs1290943_G',
 'rs741283_G',
 'rs7580907_A',
 'rs735931_T',
 'rs9848819:74123221:C:T_T',
 'rs11718972_G',
 'rs9882701_A',
 'rs9810603:184986743:T:C_C',
 'rs7629245_G',
 'rs10937477_G',
 'rs9877345_A',
 'rs4690001_G',
 'rs4054037_A',
 'rs11946808_A',
 'rs7683638_G',
 'rs12505610:171302543:G:T_T',
 'rs10434313_A',
 'rs17583962_C',
 'rs16872244_A',
 'rs1705392_C',
 'rs4976445_A',
 'rs2028268_T',
 'rs815583_T',
 'rs510420_C',
 'rs2817798_T',
 'rs41421445_C',
 'rs12531826_C',
 'rs176

### Random Forest

In [21]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -55.69183183869946
mean_absolute_error 25.43497567580952
mean_squared_error  968.0317815957836
explained_variance_score 0.10187381796567085


### Support Vector Regression

In [22]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.9340401179214342
mean_absolute_error 6.00284057179963
mean_squared_error  54.90851571126669
explained_variance_score 0.9489864488835495


# XGBoost

In [23]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -5.873291210364336
mean_absolute_error 26.06981248731737
mean_squared_error  1024.6088976078202
explained_variance_score 0.050348135679062156


## <span style="color:green"> Mutual information <span style="color:green">

In [24]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 150125
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs2454962_T     8.227665e-02
rs379387_C      7.688184e-02
rs7116167_T     7.608883e-02
rs2198207_C     7.377785e-02
rs2581623_T     7.361112e-02
                    ...     
rs9315776_G     1.771523e-06
rs11855700_T    1.480320e-06
rs2251611_G     1.455906e-06
rs8042116_T     8.360537e-07
rs12128709_G    3.290196e-07
Length: 74209, dtype: float64>
Length of mutal info non zero 74209
print shape of selected snps by Mutual_info (1282, 5000)


### Linear Regression

In [25]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [26]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(M1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
MI_LR_sigSNP
MI_LR_sig_col= list(MI_LR_sigSNP.columns)
MI_LR_sig_col

r2 is  -2.588495031070614
mean_absolute_error 30.88086267645026
mean_squared_error  1458.162651310802
explained_variance_score -0.3548642605452863
Selected p-values:
[0.00020168 0.00014395]
Number of selected features: 2
Selected F-values:
[13.89567645 14.5373109 ]
Indices of selected features with top F-values:
[1 0]
Top selected features:
      rs1412922_G  rs10692334:8486910:C:CTT_CTT
0               2                             1
1               2                             2
2               2                             1
3               2                             1
4               2                             2
...           ...                           ...
1277            2                             2
1278            2                             2
1279            2                             1
1280            2                             2
1281            2                             2

[1282 rows x 2 columns]


['rs10692334:8486910:C:CTT_CTT', 'rs1412922_G']

### Random Forest

In [27]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -68.71401794897369
mean_absolute_error 26.826803813776582
mean_squared_error  1088.4656644185425
explained_variance_score -0.010302914736393998


### Support Vector Regressor

In [28]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -2.0225961905191903
mean_absolute_error 31.411199609285006
mean_squared_error  1561.448423717936
explained_variance_score -0.4484188686735737


### XGboost

In [29]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 score -12.012368342137014
mean_absolute_error 27.327849866198253
mean_squared_error  1172.9832119347816
explained_variance_score -0.08982811053637163
